# Numbers

[Numeric types](https://go.dev/ref/spec#Numeric_types) section in [The Go Programming Language Specification](https://go.dev/ref/)

Before we start, check Go version:

In [1]:
%%
fmt.Println(runtime. Version())

go1.23.4


## Integers

Go has 8 fixed-size integer types and two widely used aliases. From the [specs](https://go.dev/ref/spec#Numeric_types):

```go
uint8       // unsigned  8-bit integers (0 to 255)
uint16      // unsigned 16-bit integers (0 to 65535)
uint32      // unsigned 32-bit integers (0 to 4294967295)
uint64      // unsigned 64-bit integers (0 to 18446744073709551615)

int8        // signed  8-bit integers (-128 to 127)
int16       // signed 16-bit integers (-32768 to 32767)
int32       // signed 32-bit integers (-2147483648 to 2147483647)
int64       // signed 64-bit integers (-9223372036854775808 to 9223372036854775807)

float32     // IEEE 754 32-bit floating-point numbers
float64     // IEEE 754 64-bit floating-point numbers

complex64   // complex numbers with float32 real and imaginary parts
complex128  // complex numbers with float64 real and imaginary parts

byte        alias for uint8
rune        alias for int32
```

Go also has 3 integer types with implementation-dependent sizes:

```go
uint     // either 32 or 64 bits
int      // same size as uint
uintptr  // an unsigned integer large enough to store the uninterpreted bits of a pointer value
```

The integer types all wrap-around: if you add 1 to the maximum value, you get the minimum value, and vice-versa.

Examples:

In [2]:
%%
var u uint8
fmt.Println(u)
fmt.Println(u - 1)

0
255


In [3]:
%%
var u uint8 = 255
fmt.Println(u)
fmt.Println(u + 1)

255
0


In [4]:
%%
var i int8 = 127
fmt.Println(i + 1)

-128


In [5]:
%%
var i int8 = -128
fmt.Println(i - 1)

127


In [6]:
func pow2(exponent uint64) (result uint64) {
	result = 1
    var i uint64
	for i = 0; i < exponent; i++ {
		result *= 2
	}
	return result
}

In [7]:
%%
fmt.Println(pow2(63))
fmt.Println(pow2(64))
fmt.Println(pow2(64)-1)
const uint64max = 18446744073709551615  // https://go.dev/ref/spec#Numeric_types
fmt.Println(pow2(64)-1 == uint64max) 

9223372036854775808
0
18446744073709551615
true


### What's the size of my int?

In [8]:
%%
fmt.Println(bits.UintSize)

64


In [9]:
%%
uintSize := 32 << (^uint(0) >> 32 & 1) // 32 or 64 
fmt.Println(uintSize)

64


In [10]:
%%
fmt.Println(^uint32(0))

4294967295


In [11]:
%%
// for i := 0; i < 9; i++ {
for i := range 10 {
    j := ^uint8(0) >> i
    fmt.Printf("%d %3d %08b\n", i, j, j)
}


0 255 11111111
1 127 01111111
2  63 00111111
3  31 00011111
4  15 00001111
5   7 00000111
6   3 00000011
7   1 00000001
8   0 00000000
9   0 00000000


## The `byte8` and `rune` aliases

## Floats

In [12]:
%%
var f float32
for f = -20; f <= 20; f++ {
    bits := fmt.Sprintf("%032b", math.Float32bits(f))
    fmt.Printf("%5.1f  %v %v %v\n", f, bits[:1], bits[1:9], bits[9:])
}

-20.0  1 10000011 01000000000000000000000
-19.0  1 10000011 00110000000000000000000
-18.0  1 10000011 00100000000000000000000
-17.0  1 10000011 00010000000000000000000
-16.0  1 10000011 00000000000000000000000
-15.0  1 10000010 11100000000000000000000
-14.0  1 10000010 11000000000000000000000
-13.0  1 10000010 10100000000000000000000
-12.0  1 10000010 10000000000000000000000
-11.0  1 10000010 01100000000000000000000
-10.0  1 10000010 01000000000000000000000
 -9.0  1 10000010 00100000000000000000000
 -8.0  1 10000010 00000000000000000000000
 -7.0  1 10000001 11000000000000000000000
 -6.0  1 10000001 10000000000000000000000
 -5.0  1 10000001 01000000000000000000000
 -4.0  1 10000001 00000000000000000000000
 -3.0  1 10000000 10000000000000000000000
 -2.0  1 10000000 00000000000000000000000
 -1.0  1 01111111 00000000000000000000000
  0.0  0 00000000 00000000000000000000000
  1.0  0 01111111 00000000000000000000000
  2.0  0 10000000 00000000000000000000000
  3.0  0 10000000 1000000000000000

In [13]:
func parse(s string) int64 {
    i, err := strconv.ParseInt(s, 2, 64)
    if err != nil {
        panic(err)
    }
    return i
}

In [14]:
%%
var i int64
for i = range(11) {
    bits := fmt.Sprintf("%b", i)
    n := parse(bits)
    fmt.Printf("%4v %2d\n", bits, n)
}

   0  0
   1  1
  10  2
  11  3
 100  4
 101  5
 110  6
 111  7
1000  8
1001  9
1010 10


In [15]:
import "math" 

func table(sample []float32) {
    var sign rune
    fmt.Println("   sample   SIGN  EXPONENT               FRACTION")
    for _, f := range sample {
        bits := math.Float32bits(f)
        bitStr := fmt.Sprintf("%032b", bits)
        if (bits >> 31) == 1 {
            sign = '-'
        } else {
            sign = '+'
        }
        signBit := bitStr[0]  
        expStr := bitStr[1:9]
        x := parse(expStr)
        exp := x - 127
        fmt.Printf("%10.5f  [%c]%c  [%v] %3d → %4d  [%v]\n", f, signBit, sign, expStr, x, exp, bitStr[9:])
    }
}

In [16]:
%%
table([]float32{-3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 31, 63, 127, 255, 511, 1023, 1024})

   sample   SIGN  EXPONENT               FRACTION
  -3.00000  [1]-  [10000000] 128 →    1  [10000000000000000000000]
  -2.00000  [1]-  [10000000] 128 →    1  [00000000000000000000000]
  -1.00000  [1]-  [01111111] 127 →    0  [00000000000000000000000]
   0.00000  [0]+  [00000000]   0 → -127  [00000000000000000000000]
   1.00000  [0]+  [01111111] 127 →    0  [00000000000000000000000]
   2.00000  [0]+  [10000000] 128 →    1  [00000000000000000000000]
   3.00000  [0]+  [10000000] 128 →    1  [10000000000000000000000]
   4.00000  [0]+  [10000001] 129 →    2  [00000000000000000000000]
   5.00000  [0]+  [10000001] 129 →    2  [01000000000000000000000]
   6.00000  [0]+  [10000001] 129 →    2  [10000000000000000000000]
   7.00000  [0]+  [10000001] 129 →    2  [11000000000000000000000]
   8.00000  [0]+  [10000010] 130 →    3  [00000000000000000000000]
   9.00000  [0]+  [10000010] 130 →    3  [00100000000000000000000]
  10.00000  [0]+  [10000010] 130 →    3  [01000000000000000000000]
  11.00000  

In [17]:
%%

sample := make([]float32, 6)
for i := range sample {
     sample[i] = float32(1 / math.Pow(2, float64(i)))
}
table(sample)

   sample   SIGN  EXPONENT               FRACTION
   1.00000  [0]+  [01111111] 127 →    0  [00000000000000000000000]
   0.50000  [0]+  [01111110] 126 →   -1  [00000000000000000000000]
   0.25000  [0]+  [01111101] 125 →   -2  [00000000000000000000000]
   0.12500  [0]+  [01111100] 124 →   -3  [00000000000000000000000]
   0.06250  [0]+  [01111011] 123 →   -4  [00000000000000000000000]
   0.03125  [0]+  [01111010] 122 →   -5  [00000000000000000000000]


In [18]:
%%
table([]float32{0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1})

   sample   SIGN  EXPONENT               FRACTION
   0.00000  [0]+  [00000000]   0 → -127  [00000000000000000000000]
   0.10000  [0]+  [01111011] 123 →   -4  [10011001100110011001101]
   0.20000  [0]+  [01111100] 124 →   -3  [10011001100110011001101]
   0.30000  [0]+  [01111101] 125 →   -2  [00110011001100110011010]
   0.40000  [0]+  [01111101] 125 →   -2  [10011001100110011001101]
   0.50000  [0]+  [01111110] 126 →   -1  [00000000000000000000000]
   0.60000  [0]+  [01111110] 126 →   -1  [00110011001100110011010]
   0.70000  [0]+  [01111110] 126 →   -1  [01100110011001100110011]
   0.80000  [0]+  [01111110] 126 →   -1  [10011001100110011001101]
   0.90000  [0]+  [01111110] 126 →   -1  [11001100110011001100110]
   1.00000  [0]+  [01111111] 127 →    0  [00000000000000000000000]


The set of `float64` is not a superset of `int64`.

In [19]:
%%
for e := range 64 {
    var i = 1 << e - 1
    var f = float64(i)
    var is = fmt.Sprintf("%d", i)
    var fs = fmt.Sprintf("%.f", f)
    if is != fs {
        fmt.Printf("%d\t%d\n\t%.1f\n", e, i, f)
    }
}


54	18014398509481983
	18014398509481984.0
55	36028797018963967
	36028797018963968.0
56	72057594037927935
	72057594037927936.0
57	144115188075855871
	144115188075855872.0
58	288230376151711743
	288230376151711744.0
59	576460752303423487
	576460752303423488.0
60	1152921504606846975
	1152921504606846976.0
61	2305843009213693951
	2305843009213693952.0
62	4611686018427387903
	4611686018427387904.0
63	9223372036854775807
	9223372036854775808.0


## Complex numbers

The types `complex64` and `complex128` represent complex numbers, represented as pairs of floats, as implemented in modern CPUs. 